In [1]:
import os
import numpy as np

In [2]:
print(os.listdir(os.getcwd()))
print(os.getcwd())

['.ipynb_checkpoints', '2_d_blood_vessel_pnts.npy', '2_d_regular_brain_pnts.npy', '2_d_tumor_pnts.npy', 'AutoEncoder.ipynb', 'bailey', 'blood_vessels_data_pts.npy', 'blood_vessel_ts.npy', 'chessher', 'data_accumulator.ipynb', 'decision_tree.ipynb', 'Edmondson', 'finch', 'Garris', 'gostkowski', 'Gryder', 'Larsson', 'LSM.ipynb', 'lstm_blood_vessel_output.npy', 'lstm_regular_brain_output.npy', 'lstm_tumor_output.npy', 'marks', 'mccall', 'milligan', 'playground.ipynb', 'regular_brain_ts.npy', 'sharp', 'spikeView.py', 'stevens', 'Tumors', 'tumor_data_pts.npy', 'tumor_lsm_output.npy', 'tumor_ts.npy', 'underwood', 'Untitled.ipynb']
C:\Users\Cmorgan1\data_aquisition


## Function to collect specific type of data 

In [3]:
def get_numpy_arr_of_type(input_type, path = ''):
    if path == '':
        path = os.getcwd()
    folders = os.listdir(path)
    points = np.empty((0,3), int)
    grid = np.empty((0,17,17,17), int)
    for folder in folders:
        if folder.find('.ipynb') == -1 and folder.find('.py') == -1 and folder.find('.npy') == -1:
            print(os.path.join(path, folder))
            files = os.listdir(os.path.join(path, folder))
            for file in files:
                if str(file).endswith('.npy') and file.find(input_type) != -1:
                    ar = np.load(os.path.join(path,folder, file))
                    if str(file).endswith('pts.npy'):
                        points = np.vstack([points,ar])
                    else:
                        grid = np.vstack([grid, ar])
                        
    return grid, points
                    

## Function to convert to 51 by 17 format
version2d has been changed so that it is now completely vectorized 

In [4]:
import numpy as np
import matplotlib.pyplot as plt

def version1D(b1):
    c1 = (np.sum(b1, axis=0)).reshape(17*17)
    c2 = (np.sum(b1, axis=1)).reshape(17*17)
    c3 = (np.sum(b1, axis=2)).reshape(17*17)
    print(c1.shape)

    # why not np.rint ? (what exactly is this doing)
    c1 = (1-0)*((c1-np.min(c1))/(np.max(c1) - np.min(c1)))
    c2 = (1-0)*((c2-np.min(c2))/(np.max(c2) - np.min(c2)))
    c3 = (1-0)*((c3-np.min(c3))/(np.max(c3) - np.min(c3)))
    
    #d = b1.reshape(17*17*17)
    # why 306? 
    p = np.arange(0,306,17)
    spikes = np.zeros((3, 17))
    ave = np.mean(c1)
    std1 = np.std(c1)

    for t in range(17):
        max1 = np.max(c1[p[t]:p[t+1]])
        max2 = np.max(c2[p[t]:p[t+1]])
        max3 = np.max(c3[p[t]:p[t+1]])
        if max1 > 0.65: #ave + 1*std1:
            spikes[0,t] = 1
        if max2 > 0.65: #ave + 1*std1:
            spikes[1,t] = 1
        if max2 > 0.65: #ave + 1*std1:
            spikes[2,t] = 1
    print(spikes)

    #plt.plot(d)
    #plt.show()

    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, sharex=True)
    ax1.plot(c1)
    ax2.plot(c2)
    ax3.plot(c3)
    plt.show()

def version2D(b1, threshold):
    c1 = (np.sum(b1, axis=0))
    c2 = (np.sum(b1, axis=1))
    c3 = (np.sum(b1, axis=2))

    c1 = (1-0)*((c1-np.min(c1))/(np.max(c1) - np.min(c1)))
    c2 = (1-0)*((c2-np.min(c2))/(np.max(c2) - np.min(c2)))
    c3 = (1-0)*((c3-np.min(c3))/(np.max(c3) - np.min(c3)))

    d1 = np.zeros((17,17))
    d2 = np.zeros((17,17))
    d3 = np.zeros((17,17))
    
    c1 -= (threshold - .5)
    c2 -= (threshold - .5)
    c3 -= (threshold - .5)
    
    d1 = np.rint(c1)
    d2 = np.rint(c2)
    d3 = np.rint(c3)

    d4 = np.vstack([d1,d2,d3])
    return d4

def showResults(dname):

    im = np.load('data/'+dname)
    plt.imshow(im)
    plt.show()


"""if __name__ == '__main__':

    threshold = 0.7
    num1 = 2
#    a = np.load('../tumorBrain_M_17/test_17_patient_underwood_test2.npy')
#    b1 = a[num1,:,:,:]
#    out1 = version2D(b1, threshold)
#    np.save('data/underwood2_array' + str(num1) + '.npy', out1) 
    dname = 'marks_array' + str(num1) + '.npy'
    showResults(dname)"""


"if __name__ == '__main__':\n\n    threshold = 0.7\n    num1 = 2\n#    a = np.load('../tumorBrain_M_17/test_17_patient_underwood_test2.npy')\n#    b1 = a[num1,:,:,:]\n#    out1 = version2D(b1, threshold)\n#    np.save('data/underwood2_array' + str(num1) + '.npy', out1) \n    dname = 'marks_array' + str(num1) + '.npy'\n    showResults(dname)"

## Functions to interact with output of version2d

In [5]:
def visualize_data(points,  thresh, area):
    for i in range(points.shape[0]):
        print(i)
        for x in area:
            print(thresh + x)
            data = version2D(points[i], thresh + x)
            plt.imshow(data)
            plt.show()
        print() 

def get_rotations(points):
    #computes 4 rotations of 90 degree turns around the z axis
    results = []
    for arr in points:
        for i in range(4):
            new_arr = np.rot90(arr, i, [1,2])
            results.append(new_arr)
    return np.array(results)

def convert_2d(ar_points, threshold):
    results = []
    for points in ar_points:
        results.append(version2D(points, threshold))
    return np.array(results)

In [18]:
points, grid = get_numpy_arr_of_type("blood", path = '')

C:\Users\Cmorgan1\data_aquisition\bailey
C:\Users\Cmorgan1\data_aquisition\chessher
C:\Users\Cmorgan1\data_aquisition\Edmondson
C:\Users\Cmorgan1\data_aquisition\finch
C:\Users\Cmorgan1\data_aquisition\Garris
C:\Users\Cmorgan1\data_aquisition\gostkowski
C:\Users\Cmorgan1\data_aquisition\Gryder
C:\Users\Cmorgan1\data_aquisition\Larsson
C:\Users\Cmorgan1\data_aquisition\marks
C:\Users\Cmorgan1\data_aquisition\mccall
C:\Users\Cmorgan1\data_aquisition\milligan
C:\Users\Cmorgan1\data_aquisition\sharp
C:\Users\Cmorgan1\data_aquisition\stevens
C:\Users\Cmorgan1\data_aquisition\Tumors
C:\Users\Cmorgan1\data_aquisition\underwood


In [19]:
## cut off is set at .75
rotations = get_rotations(points)
results = convert_2d(rotations, .75)

In [21]:
np.save("2_d_new_blood_vessel_pnts.npy", results)